# Download the IMDB Dataset

In [1]:
# Download reviews.txt and labels.txt from here: https://github.com/udacity/deep-learning/tree/master/sentiment-network

def pretty_print_review_and_label(i):
   print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('reviews.txt','r') # What we know!
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

g = open('labels.txt','r') # What we WANT to know!
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()

# Capturing Word Correlation in Input Data

In [2]:
import numpy as np

onehots = {}
onehots['cat'] = np.array([1,0,0,0])
onehots['the'] = np.array([0,1,0,0])
onehots['dog'] = np.array([0,0,1,0])
onehots['sat'] = np.array([0,0,0,1])

sentence = ['the','cat','sat']
x = onehots[sentence[0]] + \
    onehots[sentence[1]] + \
    onehots[sentence[2]]

print("Sent Encoding:" + str(x))

Sent Encoding:[1 1 0 1]


# Predicting Movie Reviews

In [3]:
import sys

# 1) Reading the movie reviews
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

# 'raw_reviews' is now a list of strings, each string is a movie review line.

# 2) Reading the labels (positive/negative)
f = open('labels.txt')
raw_labels = f.readlines()
f.close()

# 'raw_labels' is now a list of strings, each string is either
# 'positive\n' or 'negative\n'.

# 3) Tokenizing reviews: split each review on spaces to get individual words
#    then convert that list to a set (removing duplicates).
tokens = list(map(lambda x: set(x.split(" ")), raw_reviews))

# Now 'tokens' is a list of sets of words:
# Example: tokens[i] might be {"this", "movie", "was", "awesome\n"}

# 4) Build a global vocabulary of all unique words across all reviews
vocab = set()
for sent in tokens:
    for word in sent:
        if len(word) > 0:
            vocab.add(word)
# 'vocab' is now a set of all unique words encountered, ignoring empty strings.

vocab = list(vocab)
# Convert the set to a list so it has a fixed order (indexable).

# 5) Create a mapping from word to index:
#    Each unique word in 'vocab' is assigned a numeric ID.
word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i

# Example: word2index["movie"] could be 0, word2index["great"] could be 1, etc.

# 6) Build the input dataset:
#    For each review, convert the words to their indices.
#    Then store these indices as a list (removing duplicates by using set).
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            # If the word is in word2index, append its index
            sent_indices.append(word2index[word])
        except:
            # If word isn't in dictionary (shouldn't happen if vocab built properly),
            # ignore it.
            pass
    # Convert 'sent_indices' to a set to remove duplicates,
    # then back to a list to store in input_dataset.
    input_dataset.append(list(set(sent_indices)))

# 'input_dataset' is now a list of lists:
# e.g. input_dataset[i] = [23, 45, 122] means that review i
# has words with these indices in the vocabulary.

# 7) Build the target dataset from labels.txt:
target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

# So for each review, if 'labels.txt' has "positive\n", we store 1,
# otherwise 0 for negative.

# 'target_dataset' is| now a list of 1s and 0s, parallel to 'input_dataset'.


In [4]:
import numpy as np
import sys

np.random.seed(1)  # Fix seed for reproducibility

def sigmoid(x):
    # The sigmoid activation function: outputs values in (0,1).
    # For input x, output = 1 / (1 + e^-x).
    return 1 / (1 + np.exp(-x))

alpha, iterations = (0.01, 2)
# alpha = learning rate
# iterations = how many times we'll pass through the training set

hidden_size = 100
# The dimensionality of our "hidden representation" or embedding.

# weights_0_1 is an embedding matrix of shape (vocab_size, hidden_size).
#   Each row corresponds to a word in the vocabulary,
#   each column is a dimension in the embedding space.
weights_0_1 = 0.2*np.random.random((len(vocab), hidden_size)) - 0.1

# weights_1_2 is a matrix of shape (hidden_size, 1).
#   This connects the hidden embedding vector to the final output (one number).
weights_1_2 = 0.2*np.random.random((hidden_size, 1)) - 0.1

correct, total = (0, 0)  # We'll track training accuracy during training.

for iter in range(iterations):
    # We'll do multiple epochs; in this code, just 2 epochs (iterations=2).
    
    # Train on first (len(input_dataset)-1000) reviews
    for i in range(len(input_dataset) - 1000):

        x = input_dataset[i]    # Indices of words in the i-th review.
        y = target_dataset[i]   # 0 or 1 for the i-th review's label.

        # 1) EMBEDDING LOOKUP + SUM
        # weights_0_1[x] returns the embeddings for all words in the review,
        # shape (#words_in_review, hidden_size).
        # Summation across axis=0 => we get a single vector of shape (hidden_size,).
        layer_1 = sigmoid(np.sum(weights_0_1[x], axis=0))
        # We apply sigmoid after summing, so each dimension is between (0,1).

        # 2) PREDICTION LAYER
        # Dot product: (hidden_size,) dot (hidden_size,1) => scalar => apply sigmoid.
        layer_2 = sigmoid(np.dot(layer_1, weights_1_2))

        # 3) CALCULATE ERROR
        # Compare prediction (layer_2) to target y.
        layer_2_delta = layer_2 - y  # If y=1, want layer_2 close to 1; if y=0, want layer_2 close to 0.

        # 4) BACKPROP TO HIDDEN
        # We push error back through weights_1_2: shape => (1,) dot (1, hidden_size)
        # but actually we do a simpler approach: (layer_2_delta * weights_1_2^T).
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)
        # This gives how each hidden dimension contributed to the final error.

        # 5) UPDATE WEIGHTS
        # a) Update embedding matrix weights_0_1 for each word in x
        # We do 'weights_0_1[x] -= layer_1_delta * alpha' in place for all words in the review.
        #   This spreads the gradient to each word's embedding that participated in the sum.
        weights_0_1[x] -= layer_1_delta * alpha

        # b) Update weights_1_2
        # We'll do an outer product of (layer_1) (shape (hidden_size,))
        # with (layer_2_delta) (shape (1,)) => shape (hidden_size, 1)
        weights_1_2 -= np.outer(layer_1, layer_2_delta) * alpha

        # 6) TRACK TRAINING ACCURACY
        # If (layer_2 - y) is < 0.5 in absolute value, we consider it a correct guess.
        # (This is a somewhat unusual measure, but used here as a rough approximation.)
        if np.abs(layer_2_delta) < 0.5:
            correct += 1
        total += 1

        # Show progress every 10 reviews
        if (i % 10) == 9:
            # progress is fraction of how far we are through the training portion
            progress = str(i / float(len(input_dataset)))
            sys.stdout.write(
                '\rIter:' + str(iter)
                + ' Progress:' + progress[2:4] + '.' + progress[4:6] + '%'
                + ' Training Accuracy:' + str(correct / float(total)) + '%'
            )
    print()

# 7) AFTER TRAINING, EVALUATE ON LAST 1000 REVIEWS
correct, total = (0, 0)
for i in range(len(input_dataset) - 1000, len(input_dataset)):

    x = input_dataset[i]  # word indices for this test review
    y = target_dataset[i] # 0 or 1 for test label

    # FORWARD PASS ONLY (no training updates)
    layer_1 = sigmoid(np.sum(weights_0_1[x], axis=0))
    layer_2 = sigmoid(np.dot(layer_1, weights_1_2))

    # If the difference between pred and label is < 0.5, call it correct
    if np.abs(layer_2 - y) < 0.5:
        correct += 1
    total += 1

print("Test Accuracy:" + str(correct / float(total)))


Iter:0 Progress:95.99% Training Accuracy:0.8323333333333334%%
Iter:1 Progress:95.99% Training Accuracy:0.8661875%193582%7%
Test Accuracy:0.852


In [5]:
tokens[0]

{'',
 '\n',
 '.',
 'a',
 'about',
 'adults',
 'age',
 'all',
 'and',
 'as',
 'at',
 'believe',
 'bromwell',
 'burn',
 'can',
 'cartoon',
 'classic',
 'closer',
 'comedy',
 'down',
 'episode',
 'expect',
 'far',
 'fetched',
 'financially',
 'here',
 'high',
 'i',
 'immediately',
 'in',
 'insightful',
 'inspector',
 'is',
 'isn',
 'it',
 'knew',
 'lead',
 'life',
 'line',
 'm',
 'many',
 'me',
 'much',
 'my',
 'of',
 'one',
 'other',
 'pathetic',
 'pettiness',
 'pity',
 'pomp',
 'profession',
 'programs',
 'ran',
 'reality',
 'recalled',
 'remind',
 'repeatedly',
 'right',
 's',
 'sack',
 'same',
 'satire',
 'saw',
 'school',
 'schools',
 'scramble',
 'see',
 'situation',
 'some',
 'student',
 'students',
 'such',
 'survive',
 't',
 'teachers',
 'teaching',
 'than',
 'that',
 'the',
 'their',
 'think',
 'through',
 'time',
 'to',
 'tried',
 'welcome',
 'what',
 'when',
 'which',
 'who',
 'whole',
 'years',
 'your'}

# Comparing Word Embeddings

In [9]:
from collections import Counter
import math 

def similar(target='beautiful'):
    # 'target' is the word we want to find neighbors (similar words) for.
    # We'll compare all other words to this target.

    target_index = word2index[target]
    # Lookup the integer index for the target word from 'word2index'.
    # 'word2index' maps each word to its index in 'weights_0_1'.

    scores = Counter()
    # We'll keep track of each word's similarity score in a Counter.
    # A Counter in Python lets us easily retrieve top results with 'most_common'.

    for word, index in word2index.items():
        # Loop through each word in the vocabulary, along with its integer index.
        
        raw_difference = weights_0_1[index] - weights_0_1[target_index]
        # Subtract the target word's embedding from the current word's embedding.
        # This yields a vector representing the difference between the two embeddings.

        squared_difference = raw_difference * raw_difference
        # Elementwise square of that difference vector.

        # Euclidean distance = sqrt of the sum of squared differences.
        # We'll calculate it and then store the NEGATIVE distance in 'scores'.
        # Why negative? Because Counter.most_common() sorts by largest value,
        # so storing negative distance effectively makes 'closest' words rank first.
        scores[word] = -math.sqrt(sum(squared_difference))

    # Return the top 10 entries from the Counter (the words with smallest distance).
    # 'most_common(10)' will give us the words with the "highest" (least negative)
    # values, i.e., the smallest Euclidean distance from the target.
    return scores.most_common(10)


In [7]:
print(similar('beautiful'))

[('beautiful', -0.0), ('cry', -0.70018246771713), ('entertaining', -0.7042909118877329), ('beautifully', -0.7533394067175685), ('simple', -0.7570735834153209), ('shows', -0.7614350096609824), ('favorite', -0.7631952942752108), ('fun', -0.7687566747677717), ('realistic', -0.7690851907215188), ('surprisingly', -0.7710630434834971)]


In [8]:
print(similar('terrible'))

[('terrible', -0.0), ('lacks', -0.7401188100674893), ('fails', -0.7779077291892439), ('annoying', -0.7933438656652453), ('boring', -0.8071440180566011), ('disappointment', -0.80910574818041), ('worse', -0.8402021942764264), ('dull', -0.8488233841444136), ('horrible', -0.8491939210237277), ('laughable', -0.8562760637276858)]


# Filling in the Blank

In [10]:
import sys, random, math
from collections import Counter
import numpy as np

# Fix random seeds for reproducible results
np.random.seed(1)
random.seed(1)

# 1) READ THE RAW REVIEW TEXT
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

# 2) TOKENIZE REVIEWS
# Each line in 'raw_reviews' is split on spaces, forming a list of lists of tokens.
tokens = list(map(lambda x: (x.split(" ")), raw_reviews))

# 3) BUILD A COUNTER OF WORD FREQUENCIES
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        # We do 'wordcnt[word] -= 1' which effectively means the more frequent the word,
        # the larger the negative number (since we subtract).
        # It's unusual to do negative counts, but it might be for a sorting trick:
        # words with highest absolute count appear first when we do most_common().
        wordcnt[word] -= 1

# 4) CREATE A VOCABULARY LIST
# 'most_common()' gives pairs (word, count) sorted by count in ascending order
# (because we used negative counts).
vocab = list(set(map(lambda x: x[0], wordcnt.most_common())))

# 5) BUILD A MAPPING FROM WORD TO INDEX
word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i

# 6) CREATE 'input_dataset' AND 'concatenated'
#    'input_dataset' will be a list of lists (each containing word indices for one review).
#    'concatenated' is a single array of all word indices in all reviews, for negative sampling.
concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            # We also append to 'concatenated' to keep track of all word indices.
            concatenated.append(word2index[word])
        except:
            # If for some reason the word isn't in 'word2index' (shouldn't happen if built correctly), ignore it.
            pass
    input_dataset.append(sent_indices)

# Convert 'concatenated' to a NumPy array and shuffle 'input_dataset' to randomize the order of reviews.
concatenated = np.array(concatenated)
random.shuffle(input_dataset)

# 7) SET HYPERPARAMETERS
alpha, iterations = (0.05, 2)        # Learning rate and how many passes (epochs)
hidden_size, window, negative = (50, 2, 5)
# hidden_size: dimension of the hidden layer / word embeddings
# window: how many words on the left/right to consider as context
# negative: how many negative samples to draw for each true target

# 8) INITIALIZE WEIGHTS
# weights_0_1: shape (vocab_size, hidden_size). Each row is an embedding for one word.
weights_0_1 = (np.random.rand(len(vocab), hidden_size) - 0.5) * 0.2

# weights_1_2: shape (vocab_size, hidden_size). 
# For negative sampling approach, we store a vector for each word in the output space as well.
# Initialized to zero here.
weights_1_2 = np.random.rand(len(vocab), hidden_size) * 0

# 9) PREPARE THE TARGET VECTOR FOR NEGATIVE SAMPLING
# layer_2_target has length = negative + 1 => e.g. 6 total if negative=5
# The first element = 1 (the true word), the rest = 0 (the negative samples).
layer_2_target = np.zeros(negative + 1)
layer_2_target[0] = 1

# 10) SIMILARITY FUNCTION
# Returns the 10 closest words to 'target' in embedding space (Euclidean distance).
def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word, index in word2index.items():
        raw_difference = weights_0_1[index] - weights_0_1[target_index]
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
    return scores.most_common(10)

# 11) DEFINE SIGMOID ACTIVATION
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 12) TRAINING LOOP
# We'll iterate over input_dataset multiple times (in total 'iterations' times).
for rev_i, review in enumerate(input_dataset * iterations):

    # For each word in 'review', we will pick it as the "missing" word (target).
    for target_i in range(len(review)):

        # a) SELECT NEGATIVE SAMPLES
        # 'target_samples' = the true word's index + some random words (negative samples).
        target_samples = [review[target_i]] + list(
            concatenated[(np.random.rand(negative) * len(concatenated)).astype('int').tolist()]
        )

        # b) BUILD CONTEXT
        #   left_context are the words in the 'window' to the left,
        #   right_context are the words in the 'window' to the right.
        left_context = review[max(0, target_i - window) : target_i]
        right_context = review[target_i + 1 : min(len(review), target_i + window)]

        # c) FORWARD PASS
        #   1) layer_1 = mean of the embeddings of context words from weights_0_1
        layer_1 = np.mean(weights_0_1[left_context + right_context], axis=0)

        #   2) layer_2 = sigmoid( layer_1 dot weights_1_2 for each sample in target_samples )
        #   shape: (negative+1,) => first index is the true target word
        layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))

        # d) BACKPROP
        #   1) layer_2_delta = (predicted - target). target is [1,0,0,0,0,...]
        layer_2_delta = layer_2 - layer_2_target

        #   2) layer_1_delta: push error back to 'layer_1' via weights_1_2
        layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

        # e) WEIGHT UPDATES
        #   1) Update embeddings for context words
        #      Subtract the error gradient from each context word's embedding.
        weights_0_1[left_context + right_context] -= layer_1_delta * alpha

        #   2) Update weights_1_2 for each sampled word
        #      outer product of (layer_2_delta) and (layer_1).
        #      shape: (negative+1, hidden_size)
        weights_1_2[target_samples] -= np.outer(layer_2_delta, layer_1) * alpha

    # Print progress every 250 reviews
    if (rev_i % 250) == 0:
        sys.stdout.write(
            '\rProgress:' + str(rev_i / float(len(input_dataset) * iterations))
            + "   " + str(similar('terrible'))
        )
    # Also print progress after each review
    sys.stdout.write(
        '\rProgress:' + str(rev_i / float(len(input_dataset) * iterations))
    )

# After training, show the 10 most similar words to 'terrible'.
print(similar('terrible'))


Progress:0.36714 [('terrible', -0.0), ('horrible', -3.141910035029569), ('brilliant', -3.672210219060791), ('superb', -3.925677405794538), ('lame', -4.059408572809917), ('fine', -4.129043029429027), ('laughable', -4.16314118653493), ('fantastic', -4.175484995853935), ('pathetic', -4.270327456356103), ('bad', -4.279061942789954)]250625)])]1440742)]7280948)])]

KeyboardInterrupt: 

In [ ]:
def analogy(positive=['terrible','good'],negative=['bad']):
    
    norms = np.sum(weights_0_1 * weights_0_1,axis=1)
    norms.resize(norms.shape[0],1)
    
    normed_weights = weights_0_1 * norms
    
    query_vect = np.zeros(len(weights_0_1[0]))
    for word in positive:
        query_vect += normed_weights[word2index[word]]
    for word in negative:
        query_vect -= normed_weights[word2index[word]]
    
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - query_vect
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
        
    return scores.most_common(10)[1:]

In [71]:
analogy(['terrible','good'],['bad'])

[('terrific', -210.46593317724228),
 ('perfect', -210.52652806032205),
 ('worth', -210.53162266358495),
 ('good', -210.55072184482773),
 ('terrible', -210.58429046605724),
 ('decent', -210.87945442008805),
 ('superb', -211.01143515971094),
 ('great', -211.1327058081335),
 ('worthy', -211.13577238103477)]

In [72]:
analogy(['elizabeth','he'],['she'])

[('simon', -193.82490698964878),
 ('obsessed', -193.91805919583555),
 ('stanwyck', -194.22311983847902),
 ('sandler', -194.22846640800597),
 ('branagh', -194.24551334589853),
 ('daniel', -194.24631020485714),
 ('peter', -194.29908544092078),
 ('tony', -194.31388897167716),
 ('aged', -194.35115773165094)]